<a href="https://colab.research.google.com/github/DebajyotiBindu/ML-model/blob/main/Fake_news_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d algord/fake-news

Dataset URL: https://www.kaggle.com/datasets/algord/fake-news
License(s): CC0-1.0
  0% 0.00/1.68M [00:00<?, ?B/s]
100% 1.68M/1.68M [00:00<00:00, 937MB/s]


In [ ]:
!unzip fake-news.zip

Archive:  fake-news.zip
  inflating: FakeNewsNet.csv         


In [ ]:
import pandas as pd
data=pd.read_csv('/content/FakeNewsNet.csv')

In [ ]:
data.head()

,title,news_url,source_domain,tweet_num,real
0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,42,1
1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,0,1
2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,63,1
3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,20,1
4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,38,1


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23196 entries, 0 to 23195
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   title          23196 non-null  object
 1   news_url       22866 non-null  object
 2   source_domain  22866 non-null  object
 3   tweet_num      23196 non-null  int64 
 4   real           23196 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 906.2+ KB


In [ ]:
data.isna().sum()

,0
title,0
news_url,330
source_domain,330
tweet_num,0
real,0


In [ ]:
data.shape

(23196, 5)

In [ ]:
#Dropping null values
data=data.dropna()

In [ ]:
data.isna().sum()

,0
title,0
news_url,0
source_domain,0
tweet_num,0
real,0


In [ ]:
data.shape

(22866, 5)

In [ ]:
data=data.drop(['news_url'],axis=1)

In [ ]:
data.head()

,title,source_domain,tweet_num,real
0,Kandi Burruss Explodes Over Rape Accusation on...,toofab.com,42,1
1,People's Choice Awards 2018: The best red carp...,www.today.com,0,1
2,Sophia Bush Sends Sweet Birthday Message to 'O...,www.etonline.com,63,1
3,Colombian singer Maluma sparks rumours of inap...,www.dailymail.co.uk,20,1
4,Gossip Girl 10 Years Later: How Upper East Sid...,www.zerchoo.com,38,1


In [ ]:
data['real'].value_counts()

,count
real,
1,17371
0,5495


In [ ]:
X=data.drop(['real'],axis=1)
y=data['real']

In [ ]:
X.shape

(22866, 3)

In [ ]:
y.shape

(22866,)

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

In [ ]:
#vocabulary size
voc_size=5000

In [ ]:
messages=X.copy()

In [ ]:
messages['title'][1]

"People's Choice Awards 2018: The best red carpet looks"

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]
for i in range(0,len(messages)):
  review=re.sub('[^a-zA-Z]', ' ',messages['title'].iloc[i])
  review=review.lower()
  review=review.split()
  review=[ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
  review=' '.join(review)
  corpus.append(review)

In [ ]:
corpus

['kandi burruss explod rape accus real housew atlanta reunion video',
 'peopl choic award best red carpet look',
 'sophia bush send sweet birthday messag one tree hill co star hilari burton breyton eva',
 'colombian singer maluma spark rumour inappropri relationship aunt',
 'gossip girl year later upper east sider shock world chang pop cultur forev',
 'gwen stefani got dump blake shelton jealousi drama exclus',
 'broward counti sheriff fire lie parkland',
 'amber rose shut french montana date rumor call rapper bruvaaa',
 'mindi kale make first post babi appear disneyland wrinkl time co star',
 'katharin mcphee butcher toni nomin drink',
 'wag miami star ashley nicol robert philip wheeler marri',
 'mel gibson hollywood pedophil nowher left hide',
 'medium tyler henri address chill messag kristin cavallari deceas brother express read',
 'dwt season result week disney night',
 'reason tarek el moussa overcom latest back injuri',
 'david cassidi cut estrang daughter kati complet leav son b

In [ ]:
corpus[1]

'peopl choic award best red carpet look'

In [ ]:
one_hot_repr=[one_hot(words,voc_size)for words in corpus]
one_hot_repr

[[4637, 3018, 20, 1529, 147, 4418, 1293, 3787, 2765, 3483],
 [3012, 4724, 341, 1924, 1903, 2535, 2258],
 [429,
  3928,
  1049,
  2238,
  2953,
  623,
  1403,
  2049,
  1657,
  4604,
  2824,
  1810,
  3081,
  1463,
  4500],
 [462, 2097, 1119, 1006, 3010, 1990, 652, 2394],
 [816, 2398, 4741, 1578, 4373, 4123, 4786, 2082, 886, 1084, 2713, 161, 3213],
 [1028, 713, 3157, 1681, 1626, 4953, 2425, 4983, 3002],
 [2375, 4221, 3681, 4771, 2751, 1410],
 [4127, 2118, 577, 4928, 4150, 446, 3425, 2162, 71, 2399],
 [96, 576, 3122, 1097, 1596, 3535, 241, 2678, 4374, 2102, 4604, 2824],
 [4955, 1472, 1004, 1609, 1339, 4642],
 [1641, 3692, 2824, 658, 1774, 1815, 3189, 2718, 786],
 [3207, 1914, 28, 1519, 420, 2032, 1473],
 [4195, 3199, 2498, 2035, 3501, 623, 1067, 2704, 1562, 2779, 3519, 308],
 [3710, 1026, 445, 2940, 4428, 1669],
 [2675, 3831, 4259, 4270, 1843, 4526, 3764, 3238],
 [1955, 1040, 707, 2561, 4731, 2376, 1248, 2939, 4445, 308],
 [4150, 4476, 472, 3445, 3113, 2833, 226],
 [182, 3699, 4461, 1206

In [ ]:
one_hot_repr[1]

[3012, 4724, 341, 1924, 1903, 2535, 2258]

In [ ]:
sent_length=20
embedding_doc=pad_sequences(one_hot_repr,padding='post',maxlen=sent_length)
embedding_doc

array([[4637, 3018,   20, ...,    0,    0,    0],
       [3012, 4724,  341, ...,    0,    0,    0],
       [ 429, 3928, 1049, ...,    0,    0,    0],
       ...,
       [4211, 3361, 2757, ...,    0,    0,    0],
       [3110,    9,  408, ...,    0,    0,    0],
       [3765, 4990, 2939, ...,    0,    0,    0]], dtype=int32)

In [ ]:
embedding_doc[0]

array([4637, 3018,   20, 1529,  147, 4418, 1293, 3787, 2765, 3483,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [ ]:
embedding_doc[1]

array([3012, 4724,  341, 1924, 1903, 2535, 2258,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [ ]:
from tensorflow.keras.layers import Dropout
vector_features=40 #feature representation
model=Sequential()
model.add(Embedding(voc_size,vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100)) #100 is the number of neurons or memory cells
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc'])
print(model.summary())

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
len(embedding_doc),y.shape

(22866, (22866,))

In [ ]:
import numpy as np

In [ ]:
X_final=np.array(embedding_doc)
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

((22866, 20), (22866,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_final,y_final,test_size=0.33,random_state=42)

In [ ]:
X_train.shape,X_test.shape

((15320, 20), (7546, 20))

In [ ]:
y_train.shape,y_test.shape

((15320,), (7546,))

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - acc: 0.7677 - loss: 0.5414 - val_acc: 0.8341 - val_loss: 0.4014
Epoch 2/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - acc: 0.8458 - loss: 0.3630 - val_acc: 0.8296 - val_loss: 0.4294
Epoch 3/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - acc: 0.8663 - loss: 0.3253 - val_acc: 0.8272 - val_loss: 0.4111
Epoch 4/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - acc: 0.8816 - loss: 0.2944 - val_acc: 0.8212 - val_loss: 0.4370
Epoch 5/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - acc: 0.8860 - loss: 0.2763 - val_acc: 0.8204 - val_loss: 0.4832
Epoch 6/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - acc: 0.8897 - loss: 0.2712 - val_acc: 0.8065 - val_loss: 0.4713
Epoch 7/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - acc: 0.8955 - loss: 0.2551 - val_acc: 0.8060 - val_loss: 0.4457
Epoch 8/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - acc: 0.9011 - loss: 0.2506 - val_acc: 0.7988 - val_loss: 0.4924
Epoch 9/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - 

In [ ]:
y_preds=model.predict(X_test)

236/236 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


In [ ]:
y_preds=np.where(y_preds>0.6,1,0)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_preds)

array([[1195,  647],
       [ 863, 4841]])

In [ ]:
from sklearn.metrics import accuracy_score,classification_report
accuracy_score(y_test,y_preds)

0.799893983567453

In [ ]:
print(classification_report(y_test,y_preds))

              precision    recall  f1-score   support

           0       0.58      0.65      0.61      1842
           1       0.88      0.85      0.87      5704

    accuracy                           0.80      7546
   macro avg       0.73      0.75      0.74      7546
weighted avg       0.81      0.80      0.80      7546

